## CONSULTAS NO DATASET

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [23]:
DROP TABLE IF EXISTS Jogador;

CREATE TABLE Jogador (
  nome VARCHAR(50) NOT NULL,
  data_de_nascimento INTEGER NOT NULL,
  altura INT,
  mao_dominante VARCHAR(1),
  local VARCHAR(3),
  PRIMARY KEY(nome, data_de_nascimento)
) AS SELECT
  nome, data_de_nascimento, altura, mao_dominante, local
FROM CSVREAD('../data/processed/men_players.csv');

In [24]:
SELECT * FROM Jogador

In [25]:
DROP TABLE IF EXISTS Torneio;

CREATE TABLE Torneio (
  location VARCHAR(100),
  surface VARCHAR(50),
  data VARCHAR(50),
  tournament VARCHAR(100),
    
  PRIMARY KEY(tournament, data),
) AS SELECT
  Location,Surface,Data,Tournament
FROM CSVREAD('../data/processed/tournaments.csv');

In [26]:
SELECT * FROM Torneio

In [29]:
DROP TABLE IF EXISTS Confronto;

CREATE TABLE Confronto (
  winner VARCHAR(100) NOT NULL,
  loser VARCHAR(100) NOT NULL,
  round VARCHAR(100) NOT NULL,
  tournament VARCHAR(200),
  data VARCHAR(50),
  oddwinner FLOAT,
  oddloser FLOAT,
  setwinner FLOAT,
  setloser FLOAT,
  PRIMARY KEY(winner,loser,data),
  FOREIGN KEY(tournament, data) REFERENCES Torneio(tournament, data)
) AS SELECT
  Winner, Loser, Round, Tournament, Data, AvgW, AvgL,Wsets, Lsets
FROM CSVREAD('../data/processed/confrontations_since_2010.csv');

In [30]:
SELECT * FROM Confronto

In [31]:
SELECT nome FROM Jogador
WHERE mao_dominante = 'L'
AND local = 'BRA'

In [37]:
SELECT mao_dominante, COUNT(*) num 
FROM Jogador  
WHERE local = 'BRA'
GROUP BY mao_dominante

In [38]:
SELECT surface, COUNT(*) num 
FROM Torneio  
GROUP BY surface

In [62]:
DROP VIEW IF EXISTS TOP3;
CREATE VIEW TOP3 AS 
SELECT * 
FROM Confronto
WHERE winner = 'Nadal R.'
OR winner = 'Federer R.'
OR winner = 'Djokovic N.'

In [63]:
select * from top3

In [85]:
SELECT winner, TOP3.tournament, count(*)num 
FROM TOP3
GROUP BY TOP3.tournament, winner
ORDER BY num desc

In [90]:
SELECT MAX(altura)
FROM Jogador

211

In [98]:
SELECT *
FROM Jogador
WHERE altura = (SELECT MIN(altura) FROM Jogador)
OR altura = SELECT MAX(altura) FROM Jogador

In [102]:
SELECT WINNER, COUNT(*) NUM
FROM CONFRONTO
WHERE WINNER = 'Rochus O.'
OR WINNER = 'Opelka R.'
GROUP BY WINNER

In [103]:
SELECT LOSER, COUNT(*) NUM
FROM CONFRONTO
WHERE LOSER = 'Rochus O.'
OR LOSER = 'Opelka R.'
GROUP BY LOSER